# Return repo information for Neurodata using the [GitHub API](https://developer.github.com/v3/)

- Personal access token can be generated here: https://github.com/settings/tokens
  - Save token as text file in the same directory with name "TOKEN"
- Scrape the contents of the API for the repo data:
  - example response: https://developer.github.com/v3/repos/#response
  - information about getting the repo list for an organization: https://developer.github.com/v3/repos/#list-organization-repositories
  - information about getting the commits: https://developer.github.com/v3/repos/commits/
- Note that responses are capped to 100 per page.

In [1]:
import urllib
from urllib.request import urlopen
import json
import csv

with open('TOKEN','r') as myfile:
    TOKEN=myfile.read().replace('\n','')

In [2]:
with open('repo_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    objects = ['name','private','has_pages','description','pushed_at','last_commit_author',
               'last_commit_date','last_commit_email','language','forks','open_issues',
               'watchers']

    writer.writerow(objects)

    idx=1
    resp=[]
    while (len(resp)==100 or idx == 1):
        u = urlopen('https://api.github.com/orgs/neurodata/repos?page=' + str(idx) + 
                    '&per_page=100&access_token=' + TOKEN +'&sort=full_name')
        resp = json.loads(u.read().decode('utf-8'))
        #pprint(resp)
        #resp[1]["name"]

        for repo in resp:
            row=[]
            for obj in objects:
                if obj is 'last_commit_author':
                    try:
                        gu = urlopen('https://api.github.com/repos/neurodata/' + repo['name'] + 
                                     '/commits?access_token=' + TOKEN)
                        git_commits = json.loads(gu.read().decode('utf-8'))

                        #print(git_commits[0]['commit']['author']['name'])
                        #print(git_commits[0]['commit']['author']['date'])
                        row.append(git_commits[0]['commit']['author']['name'])
                        row.append(git_commits[0]['commit']['author']['date'])
                        row.append(git_commits[0]['commit']['author']['email'])
                    except urllib.error.HTTPError as err:
                        row.append(err.code)
                        row.append('')
                        row.append('')
                elif obj is not 'last_commit_date' and obj is not 'last_commit_email':
                    row.append(repo[obj])
            #print(repo["name"])
            writer.writerow(row)
        print(len(resp))
        idx += 1

100
23
